In [2]:
import sys
import pandas as pd
import seaborn as sns

sys.path.append('./../')
from src.models.continual_training import load_dataset_safe

hf_datasets = "./../data/hf_datasets"

ds_lit = load_dataset_safe(hf_datasets, "ExponentialScience/DLT-Literature")
ds_lit_dlt = load_dataset_safe(hf_datasets, "ExponentialScience/dlt-articles-ner-filtered")

# Print the first few rows of the dataset
ds_lit.select(range(5)).to_pandas().head()

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

,text,filename
0,Zhu *et al. Cybersecurity (2024) 7:12*\nh...,04a936a5b31ee9df1efed4651965f6e593bfcce6
1,2019 IEEE Symposium on Security and Privacy\n#...,17cff826c2780dc747534ae5563853c4aaf4dac7
2,## **R-SWAP: Relay based atomic cross-chain sw...,2663c9bb172e5f6e54e75edf0426d563cedbc668
3,### **Invited Paper: Fault-tolerant and Expres...,2699dde97db1b78844757db2a199456aeb20694a
4,## Automated Market Makers for Cross-chain DeF...,2a14c3302785c90aeb6aad2a0defa1bc9d92fb53


In [2]:
# Print the first few rows of the dataset
ds_lit_dlt.select(range(5)).to_pandas().head()

,text,tok_len,lang,lang_conf,predictions,num_preds,prediction_scores,prediction_max,max_pred
0,Zhu *et al. Cybersecurity (2024) 7:12*\nh...,27541,en,0.860905,"[{'end': 156, 'entity_group': 'Extensibility',...",381,"[0.5162012, 0.7531964, 0.5883707, 0.9979956, 0...",0.999950,True
1,2019 IEEE Symposium on Security and Privacy\n#...,34174,en,0.840528,"[{'end': 81, 'entity_group': 'Extensibility', ...",474,"[0.90579283, 0.50220436, 0.87687343, 0.9953546...",0.999998,True
2,## **R-SWAP: Relay based atomic cross-chain sw...,20799,en,0.875860,"[{'end': 63, 'entity_group': 'Consensus', 'sco...",338,"[0.5340054, 0.9631168, 0.9982749, 0.75958407, ...",0.999997,True
3,### **Invited Paper: Fault-tolerant and Expres...,31123,en,0.878991,"[{'end': 35, 'entity_group': 'Consensus', 'sco...",219,"[0.99317247, 0.5388175, 0.60020596, 0.99706024...",0.999930,True
4,## Automated Market Makers for Cross-chain DeF...,19083,en,0.876712,"[{'end': 76, 'entity_group': 'Consensus', 'sco...",51,"[0.9991366, 0.61591727, 0.9995122, 0.6647793, ...",0.999851,True


In [4]:
ds_lit_dlt.select(range(5)).to_pandas().columns

Index(['text', 'tok_len', 'lang', 'lang_conf', 'predictions', 'num_preds',
       'prediction_scores', 'prediction_max', 'max_pred'],
      dtype='object')

In [8]:
cols_meta = ['text', 'tok_len', 'lang', 'lang_conf', 'predictions', 'num_preds',
       'prediction_scores', 'prediction_max', 'max_pred', 'filename']

# ds_dlt = ds_lit_dlt.select(range(5))
# ds_litAll = ds_lit.select(range(100))

ds_dlt = ds_lit_dlt
ds_litAll = ds_lit

col_name = 'text'
num_processes = 26

# Create a set for cols to match  for faster lookup
left_text = set(ds_dlt[col_name])
right_text = set(ds_litAll[col_name])

filtered_litAll = ds_litAll.filter(
       lambda x: x[col_name] in left_text,
       num_proc=num_processes,
       load_from_cache_file=True
)
filtered_dlt = ds_dlt.filter(
       lambda x: x[col_name] in right_text,
       num_proc=num_processes,
       load_from_cache_file=True
)

print(f"Filtered litAll size: {len(filtered_litAll)}")
print(f"Filtered dlt size: {len(filtered_dlt)}")

Filter (num_proc=26):   0%|          | 0/147010 [00:00<?, ? examples/s]

Filter (num_proc=26):   0%|          | 0/40177 [00:00<?, ? examples/s]

Filtered litAll size: 40177
Filtered dlt size: 40177


In [4]:
filtered_litAll.to_pandas().head()

,text,filename
0,Zhu *et al. Cybersecurity (2024) 7:12*\nh...,04a936a5b31ee9df1efed4651965f6e593bfcce6
1,2019 IEEE Symposium on Security and Privacy\n#...,17cff826c2780dc747534ae5563853c4aaf4dac7
2,## **R-SWAP: Relay based atomic cross-chain sw...,2663c9bb172e5f6e54e75edf0426d563cedbc668
3,### **Invited Paper: Fault-tolerant and Expres...,2699dde97db1b78844757db2a199456aeb20694a
4,## Automated Market Makers for Cross-chain DeF...,2a14c3302785c90aeb6aad2a0defa1bc9d92fb53


In [5]:
filtered_dlt.to_pandas().head()

,text,tok_len,lang,lang_conf,predictions,num_preds,prediction_scores,prediction_max,max_pred
0,Zhu *et al. Cybersecurity (2024) 7:12*\nh...,27541,en,0.860905,"[{'end': 156, 'entity_group': 'Extensibility',...",381,"[0.5162012, 0.7531964, 0.5883707, 0.9979956, 0...",0.999950,True
1,2019 IEEE Symposium on Security and Privacy\n#...,34174,en,0.840528,"[{'end': 81, 'entity_group': 'Extensibility', ...",474,"[0.90579283, 0.50220436, 0.87687343, 0.9953546...",0.999998,True
2,## **R-SWAP: Relay based atomic cross-chain sw...,20799,en,0.875860,"[{'end': 63, 'entity_group': 'Consensus', 'sco...",338,"[0.5340054, 0.9631168, 0.9982749, 0.75958407, ...",0.999997,True
3,### **Invited Paper: Fault-tolerant and Expres...,31123,en,0.878991,"[{'end': 35, 'entity_group': 'Consensus', 'sco...",219,"[0.99317247, 0.5388175, 0.60020596, 0.99706024...",0.999930,True
4,## Automated Market Makers for Cross-chain DeF...,19083,en,0.876712,"[{'end': 76, 'entity_group': 'Consensus', 'sco...",51,"[0.9991366, 0.61591727, 0.9995122, 0.6647793, ...",0.999851,True


In [9]:
from datasets import Dataset

# sort datasets by text to ensure alignment
sorted_litAll = filtered_litAll.sort(col_name)
sorted_dlt = filtered_dlt.sort(col_name)

# Merge datasets using inner join on the 'text' column

if len(sorted_litAll) != len(sorted_dlt):
    raise ValueError("Datasets are not aligned after filtering. Lengths differ.")

new_data = {}
for col in cols_meta:
    if col in sorted_litAll.column_names:
        new_data[col] = sorted_litAll[col]
    elif col in sorted_dlt.column_names:
        new_data[col] = sorted_dlt[col]
    else:
        raise ValueError(f"Column {col} not found in either dataset.")

merged_dataset = Dataset.from_dict(new_data)

merged_dataset.to_pandas().head()

,text,tok_len,lang,lang_conf,predictions,num_preds,prediction_scores,prediction_max,max_pred,filename
0, International Journal of Management and Econ...,21192,en,0.844207,"[{'end': 878, 'entity_group': 'Blockchain_Name...",214,"[0.9804915189743042, 0.8923866748809814, 0.987...",0.999972,True,123d4ee823ce1a1a0710d7c87389a25c585ad05a
1,- ORIGINAL ARTICLE \n# Blockch...,9188,en,0.860865,"[{'end': 54, 'entity_group': 'Consensus', 'sco...",89,"[0.8663008809089661, 0.9127752184867859, 0.792...",0.999988,True,47e7c9bd25aa2a675eafcc6f792e4f529091854b
2,# International Journal of Innovat...,6872,en,0.899508,"[{'end': 832, 'entity_group': 'Consensus', 'sc...",162,"[0.5154064893722534, 0.999237060546875, 0.9772...",0.999489,True,bf311f5979ea18417370b5008edacfa5d9dec111
3,# DOCUMENTS DE TREBALL\n\n DE LA FACUL...,14278,en,0.892396,"[{'end': 1516, 'entity_group': 'ESG', 'score':...",34,"[0.9979052543640137, 0.7364181876182556, 0.564...",0.997905,True,6a37f8391f2274398a453f2b5dab80d707218b13
4,# Working Paper Series 672\n (ISSN 121...,28561,en,0.890120,"[{'end': 92, 'entity_group': 'Native_Currency_...",119,"[0.7471635341644287, 0.6694996356964111, 0.776...",0.999658,True,471d59dde619474ffeb36fbfc7f522bf1bd6a0d3


In [10]:
len(merged_dataset)

40177

In [11]:
merged_dataset.push_to_hub("ExponentialScience/DLT-Literature-PaperID", private=True)

Uploading the dataset shards:   0%|          | 0/6 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


...

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/ExponentialScience/DLT-Literature-PaperID/commit/5784b2b136c3a3da5849c0f07d8e4d2073a9f445', commit_message='Upload dataset', commit_description='', oid='5784b2b136c3a3da5849c0f07d8e4d2073a9f445', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ExponentialScience/DLT-Literature-PaperID', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ExponentialScience/DLT-Literature-PaperID'), pr_revision=None, pr_num=None)